# RTL-SDR AM Demodulator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import math
import scipy
import scipy.signal as signal
from IPython.display import Audio, display
from IPython.display import Image, SVG

In [ ]:
%config InlineBackend.figure_formats=['svg'] # SVG inline viewing

### 10 Second-captued Binary file at 135.5MHz Amplitude Modulated siganl

In [ ]:
with open('ab1355_10s.dat', mode='rb') as file: # b is important -> binary
    x = file.read()

In [ ]:
y = [(float(i)-127.5) for i in x]

In [ ]:
y_re = y[0::2]
y_im = y[1::2]

In [ ]:
len(y_re)

In [ ]:
print(y_re[100:101])

In [ ]:
plt.plot(y_re[100000:101000])

In [ ]:
plt.plot(y_re[100000:101000], y_im[100000:101000], '.--')

In [ ]:
# creating an empty object 
a= plt.figure()
axes= a.add_axes([0.1,0.1,0.8,0.8])
# adding axes
x= np.arange(0,11)
axes.plot(y_re[100000:101000], y_im[100000:101000], marker='.')
axes.set_xlim([-50,50])
axes.set_ylim([-50,50])
axes.set_aspect('equal', 'box')
plt.show()

#### y_re and y_im are lists not numpy arrays, so make them a complex numpy array

In [ ]:
y = np.zeros((len(y_re)),dtype=np.complex_)
y = np.vectorize(complex)(y_re, y_im)
#print(y)

In [ ]:
y[100:106]

In [ ]:
y.shape

In [ ]:
z = np.zeros((256, 80000),dtype=np.complex_)
Zf = np.zeros((256, 80000),dtype=np.complex_)

In [ ]:
for j in range(80000):
    for i in range(256):
        z[i][j] = y[j * 256 + i]

In [ ]:
z.shape

In [ ]:
z[99][0]

In [ ]:
z[200][70000]

In [ ]:
tmp = np.zeros(256,dtype=np.complex_)  # tmp, Tmp를 real number로 정의하면 복소수 z의 허수가 ignore됨.
Tmp = np.zeros(256,dtype=np.complex_)
Za = np.zeros((256, 80000))
for j in range(80000):
    for i in range(256):
        tmp[i] = z[i][j]
    Tmp = np.fft.fft(tmp)
    for i in range(256):
        Zf[i][j] = Tmp[i]
        Za[i][j] = np.abs(Tmp[i])

In [ ]:
print(z[3][100])
print(len(z))
Zf.shape

In [ ]:
Zf[3][100]

In [ ]:
Zf[100][20]

In [ ]:
Za.shape

In [ ]:
for i in range(256):
    Tmp[i] = np.real(Za[i][10000])

In [ ]:
Data = [[1, 2, 3], [4, 5, 6]]
plt.imshow(Data)

In [ ]:
x, y = np.mgrid[:256, :100]
plt.pcolormesh(x, y, Za[:256, 30000:30100])

In [ ]:
Tmp = np.zeros(256)
j = 10000
for i in range(256):
    Tmp[i] = np.real(Za[i][j])
plt.plot(Tmp)

In [ ]:
print(Tmp[200:210])

In [ ]:
idx = np.argmax(Tmp)
#idx = 131
#idx = 206
print(idx)

In [ ]:
plt.plot(Za[idx][:])

In [ ]:
da = np.zeros(70000)
dB = np.zeros(70000)

In [ ]:
for i in range(70000):
    da[i] = Za[idx][1000+i]

In [ ]:
mx = (max(da))
print(mx)

In [ ]:
for i in range(70000):
    dB[i] = 10* np.log10((da[i]+1e-4) / mx)

In [ ]:
Fs = 8000

In [ ]:
import sk_dsp_comm.sigsys as ss
#import sk_dsp_comm.rtlsdr_helper as sdr

In [ ]:
ss.to_wav('da.wav',Fs,da/mx) # Fs : sampling rate should be an integer number !!

In [ ]:
ls *.wav

In [ ]:
Audio('da.wav')